In [ ]:
import numpy as np
import pandas as pd

import os,shutil
import yaml
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import cv2

import torch
from sklearn.model_selection import train_test_split




In [ ]:
trainAfterNMSTable=pd.read_csv('../input/vinyolov5hyp/trainExcludeAndAfterNMS.csv')

In [ ]:
trainAfterNMSTable.info()

In [ ]:
trainAfterNMSTable.loc[:,'image_id'].value_counts()

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5,random_state=42,shuffle=True)
labels=trainAfterNMSTable['class_id']
n=skf.get_n_splits(trainAfterNMSTable,labels)
trainIndex=[]
valiIndex=[]
for i,[train_index, test_index] in enumerate(skf.split(trainAfterNMSTable, labels)):
    if i==0:
        trainIndex=train_index
        valiIndex=test_index
#trainSet.info()
#testSet.info()
trainSet=trainAfterNMSTable.iloc[trainIndex]
valiSet=trainAfterNMSTable.iloc[valiIndex]



In [ ]:
from sklearn.model_selection import GroupKFold
skf = GroupKFold(n_splits=5)
labels=trainAfterNMSTable['class_id']
groups=trainAfterNMSTable['image_id']
n=skf.get_n_splits(trainAfterNMSTable,labels,groups)
trainIndex=[]
valiIndex=[]
for i,[train_index, test_index] in enumerate(skf.split(trainAfterNMSTable, labels, groups)):
    if i==2:
        trainIndex=train_index
        valiIndex=test_index
#trainSet.info()
#testSet.info()
trainSet=trainAfterNMSTable.iloc[trainIndex]
valiSet=trainAfterNMSTable.iloc[valiIndex]

In [ ]:
trainSet.loc[:,'class_id'].value_counts()

In [ ]:
valiSet.loc[:,'class_id'].value_counts()

In [ ]:
trainSet.info()

In [ ]:
#for i in (11,0,13,3,7,8,9,10):
    #trainSet=trainSet.append(valiSet[valiSet['class_id']==i])
    #valiSet=valiSet.drop(valiSet[valiSet['class_id']==i].index)

In [ ]:
valiSet.info()

In [ ]:
valiSet.loc[:,'class_id'].value_counts()

In [ ]:
#i=0
#j=0
#for index,row in valiSet.iterrows():
#    j=j+1
#    if (row['image_id'] in trainSet['image_id'].values):
#        valiSet=valiSet.append(trainSet[trainSet['image_id']==row['image_id']])
#        trainSet=trainSet.drop(trainSet[trainSet['image_id']==row['image_id']].index)
#        i=i+1
#print(i,j)

In [ ]:
#valiSet.info()

In [ ]:
#valiSet.loc[:,'class_id'].value_counts()

In [ ]:
#valiSet.loc[:,'class_id'].value_counts()/trainSet.loc[:,'class_id'].value_counts()

In [ ]:
#trainSet.info()

In [ ]:
#from sklearn.model_selection import StratifiedKFold
os.makedirs('./vinforyolo/labels/train', exist_ok = True)
os.makedirs('./vinforyolo/labels/val', exist_ok = True)
os.makedirs('./vinforyolo/images/train', exist_ok = True)
os.makedirs('./vinforyolo/images/val', exist_ok = True)

In [ ]:
TRAIN_LABELS_PATH = './vinforyolo/labels/train'
VAL_LABELS_PATH = './vinforyolo/labels/val'
TRAIN_IMAGES_PATH = './vinforyolo/images/train'
VAL_IMAGES_PATH = './vinforyolo/images/val'

for index,row in tqdm(trainSet.iterrows()):
    fileName=row['image_id']
    boxInfo=str([int(row['class_id']),row['x_mid'],row['y_mid'],row['w'],row['h']])
    boxInfo=boxInfo.replace(",","")
    boxInfo=boxInfo.replace("[","")
    boxInfo=boxInfo.replace("]","")
    file=open(os.path.join(TRAIN_LABELS_PATH,f"{fileName}.txt"), 'a')
    #print(boxInfo)
    file.write(boxInfo)
    file.write('\n')
    file.close()
    shutil.copy(
        os.path.join('../input/vinbigdata-chest-xray-resized-png-1024x1024/train',f"{fileName}.png" ),          
        TRAIN_IMAGES_PATH
    )


for index,row in tqdm(valiSet.iterrows()):
    fileName=row['image_id']
    boxInfo=str([int(row['class_id']),row['x_mid'],row['y_mid'],row['w'],row['h']])
    boxInfo=boxInfo.replace(",","")
    boxInfo=boxInfo.replace("[","")
    boxInfo=boxInfo.replace("]","")
    file=open(os.path.join(VAL_LABELS_PATH,f"{fileName}.txt"), 'a')
    #print(boxInfo)
    file.write(boxInfo)
    file.write('\n')
    file.close()
    shutil.copy(
        os.path.join('../input/vinbigdata-chest-xray-resized-png-1024x1024/train',f"{fileName}.png" ),          
        VAL_IMAGES_PATH
    )

In [ ]:
class_ids, class_names = list(zip(*set(zip(trainAfterNMSTable.class_id, trainAfterNMSTable.class_name))))
classes = list(np.array(class_names)[np.argsort(class_ids)])
classes = list(map(lambda x: str(x), classes))
classes

In [ ]:
from os import listdir
from os.path import isfile, join
from glob import glob
import yaml

cwd = './'

with open(join( cwd , 'train.txt'), 'w') as f:
    for path in glob('./vinforyolo/images/train/*'):
        f.write('.'+path+'\n')
            
with open(join( cwd , 'val.txt'), 'w') as f:
    for path in glob('./vinforyolo/images/val/*'):
        f.write('.'+path+'\n')

data = dict(
    train =  '../train.txt',
    val   =  '../val.txt',
    nc    = 14,
    names = classes
    )

with open(join( cwd , 'vinforyolo.yaml'), 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)
    
f = open(join( cwd, 'vinforyolo.yaml'), 'r')
print('\nyaml:')
print(f.read())

In [ ]:
shutil.copytree('../input/yolov50326', '/kaggle/working/yolov5') 

In [ ]:
shutil.copy('../input/vinyolov5xgroupkfold/general.py', '/kaggle/working/yolov5/utils') 

In [ ]:
os.listdir()

In [ ]:
os.chdir('./yolov5')

In [ ]:
!pip install -r ./requirements.txt

In [ ]:
!WANDB_MODE="dryrun" python /kaggle/working/yolov5/train.py --img 1024 --batch 4 --epochs 15\
--data /kaggle/working/vinforyolo.yaml\
--cfg /kaggle/working/yolov5/models/hub/yolov5x6.yaml\
--weights yolov5x6.pt\
--hyp ../../input/vinyolov5hyp/yolov5xfromdis.yaml

In [ ]:
!WANDB_MODE="dryrun" python /kaggle/working/yolov5/train.py --img 1024 --batch 4 --epochs 10\
--data /kaggle/working/vinforyolo.yaml\
--cfg /kaggle/working/yolov5/models/hub/yolov5x6.yaml\
--weights runs/train/exp/weights/last.pt\
--hyp ../../input/vinyolov5hyp/yolov5xfromdis.yaml

# <a href="yolov5"> Download File </a>

!WANDB_MODE="dryrun" python /kaggle/working/yolov5/train.py --img 1024 --batch 4 --epochs 20\
--data /kaggle/working/vinforyolo.yaml\
--cfg /kaggle/working/yolov5/models/hub/yolov5x6.yaml\
--weights runs/train/exp/weights/last.pt\
--hyp ../../input/vinyolov5hyp/yolov5xfromdis.yaml

! find runs/train/exp |zip exp.zip -@

!WANDB_MODE="dryrun" python /kaggle/working/yolov5/train.py --img 640 --batch 16 --epochs 20\
--data /kaggle/working/vinforyolo.yaml\
--cfg /kaggle/working/yolov5/models/yolov5x.yaml\
--weights ./runs/train/exp/weights/last.pt\
--hyp ../../input/vinyolov5hyp/yolov5xfromdis.yaml

! find runs/train/exp4 |zip exp4.zip -@

!WANDB_MODE="dryrun" python /kaggle/working/yolov5/train.py --img 640 --batch 16 --epochs 20\
--data /kaggle/working/vinforyolo.yaml\
--cfg /kaggle/working/yolov5/models/yolov5x.yaml\
--weights ./runs/train/exp3/weights/last.pt\
--hyp ../../input/vinyolov5hyp/yolov5xfromdis.yaml

<a href="yolov5"> Download File </a>